In [1]:
!pip install tensorflow==2.14.0
!pip install update urllib3

!pip install textattack
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.2 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.2.0
    Uninstalling google-auth-oauthlib-1.2.0:
      Successfully uninstalled google-auth-oauthlib-1.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.1
    Uninstalling te

In [2]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification
import numpy as np
import evaluate
from datasets import load_dataset, DatasetDict
import torch
import transformers
import textattack
from textattack.models.helpers import T5ForTextToText
from textattack.models.tokenizers import T5Tokenizer
from textattack.models.wrappers.huggingface_model_wrapper import HuggingFaceModelWrapper
torch.cuda.empty_cache()

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Functions

In [3]:
metric = evaluate.load("accuracy")
tokenizer = AutoTokenizer.from_pretrained("ahmedrachid/FinancialBERT")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenizer_config.json:   0%|          | 0.00/324 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/464k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Load Dataset

In [4]:
dataset = load_dataset("financial_phrasebank", 'sentences_50agree')
test = dataset['train'].train_test_split(test_size=0.3,seed=1997)
train = test['train'].train_test_split(test_size=0.2,seed=1997)
dataset = DatasetDict({
    'train': train['train'],
    'test': test['test'],
    'validation': train['test']})
dataset.save_to_disk("content/drive/MyDrive/NLP Project/data/")

training_data = dataset['train']
validation_data = dataset['validation']
test_data = dataset['test']
tokenized_training_data = training_data.map(tokenize_function, batched=True)
tokenized_validation_data = validation_data.map(tokenize_function, batched=True)
tokenized_test_data = test_data.map(tokenize_function, batched=True)


Generating train split:   0%|          | 0/4846 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2713 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1454 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/679 [00:00<?, ? examples/s]

Map:   0%|          | 0/2713 [00:00<?, ? examples/s]

Map:   0%|          | 0/679 [00:00<?, ? examples/s]

Map:   0%|          | 0/1454 [00:00<?, ? examples/s]

Train Model

In [5]:
model = AutoModelForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("ahmedrachid/FinancialBERT")

training_args = TrainingArguments(output_dir = "test_trainer",evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_training_data,
    eval_dataset=tokenized_validation_data,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("content/drive/MyDrive/NLP Project/models/classifier_model/finbert-sentiment-no-defensive")

config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ahmedrachid/FinancialBERT and are newly initialized: ['classifier.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.392019,0.849779
2,0.422600,0.618720,0.815906
3,0.141700,0.765312,0.840943


Define attack

In [6]:
from textattack.transformations import WordSwap
from textattack.goal_functions import UntargetedClassification
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack import Attack
from textattack.search_methods import GreedySearch
import inflect

class NumberToWordSwap(WordSwap):
  def __init__(self):
    super(NumberToWordSwap, self).__init__()
    self.engine = inflect.engine()

  def _get_replacement_words(self, word):

    if word.isdigit():
      # Convert number to word
      converted = int(word) if float(word).is_integer() else float(word)

      return [self.engine.number_to_words(converted)]

    return []

def custom_attack(model_wrapper):
  goal_function = UntargetedClassification(model_wrapper)
  transformation = NumberToWordSwap()
  constraints = [RepeatModification(), StopwordModification()]
  search_method = GreedySearch()

  # Create the number to word swap attack
  return Attack(goal_function, constraints, transformation, search_method)

Attack train data

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("content/drive/MyDrive/NLP Project/models/classifier_model/finbert-sentiment-no-defensive")
model_wrapper = HuggingFaceModelWrapper(model,tokenizer)
attack = custom_attack(model_wrapper)
textattack_dataset = textattack.datasets.HuggingFaceDataset(dataset['train'])
attack_args = textattack.AttackArgs(
    num_examples=-1,
    log_to_csv="content/drive/MyDrive/NLP Project/data/attacks.csv",
)
attacker = textattack.Attacker(attack, textattack_dataset, attack_args)
attacks = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path content/drive/MyDrive/NLP Project/data/attacks_TextFoolerJin2019.csv


Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  NumberToWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 3 / 0 / 3:   0%|          | 3/2713 [00:00<12:09,  3.72it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Neutral (100%)]] --> [[[FAILED]]]

About 72 % of Evraz Group shares are owned by Lanebrook , whose beneficiaries , on the one hand , are Millhouse , the holding company for the assets of billionaire Roman Abramovich and his business partners 50 % ; and Evraz executives Alexander Abramov and Alexander Frolov 50 % , on the other .


--------------------------------------------- Result 2 ---------------------------------------------
[[Neutral (100%)]] --> [[[FAILED]]]

MANAVIGATOR-November 1 , 2010-Deal snapshot : SWEDISH PEAB BUYS EQUIPMENT RENTAL FIRM CRAMO Swedish construction company Peab AB ( STO : PEAB B ) announced the acquisition of equipment rental company Cramo Entreprenadmaskiner AB from Finnish group Cramo Oyj ( HEL : CRA1V ) .


--------------------------------------------- Result 3 ---------------------------------------------
[[Neutral (100%)]] --> [[[FAILED]]]

The compan

[Succeeded / Failed / Skipped / Total] 0 / 4 / 0 / 4:   0%|          | 4/2713 [00:01<17:34,  2.57it/s]

...

--------------------------------------------- Result 2708 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 .




[Succeeded / Failed / Skipped / Total] 14 / 2676 / 20 / 2710: 100%|█████████▉| 2711/2713 [09:32<00:00,  4.73it/s]

--------------------------------------------- Result 2709 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

Net sales grew in the period to  x20ac 402 million $ 585US million from  x20ac 401 million in 2006 .


--------------------------------------------- Result 2710 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

Major Order in India Comptel Corporation has received a significant long-term order for mediation and provisioning solutions being used by a leading operator in India .




[Succeeded / Failed / Skipped / Total] 14 / 2679 / 20 / 2713: 100%|██████████| 2713/2713 [09:32<00:00,  4.74it/s]

--------------------------------------------- Result 2711 ---------------------------------------------
[[Neutral (100%)]] --> [[[FAILED]]]

The first charging stations will be made available already in 2008 and a whole network is to be ready in 2010 .


--------------------------------------------- Result 2712 ---------------------------------------------
[[Neutral (100%)]] --> [[[FAILED]]]

The Costanza light , with an aluminum base and washable shade , comes in white , pistachio , orange , blue and red .


--------------------------------------------- Result 2713 ---------------------------------------------
[[Neutral (100%)]] --> [[[FAILED]]]

The combined capital of these funds is expected to be EUR 100mn-150mn .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 14     |
| Number of failed attacks:     | 2679   |
| Number of skipped attacks:    | 20     |
| Original 

Load perturbed data

In [10]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd
df = pd.read_csv("/content/content/drive/MyDrive/NLP Project/data/attacks.csv")

df = pd.DataFrame(df)

df = pd.read_csv("/content/content/drive/MyDrive/NLP Project/data/attacks.csv")
df = pd.DataFrame(df)
orgininal_training_data = df[["original_text","ground_truth_output"]]
orgininal_training_data.rename(columns={"original_text":"sentence","ground_truth_output":"label"},inplace=True)
perturbed_training_data = df[df["result_type"]=="Successful"][["perturbed_text","ground_truth_output"]]
perturbed_training_data.rename(columns={"perturbed_text":"sentence","ground_truth_output":"label"}, inplace=True)
perturbed_training_data = pd.concat([orgininal_training_data,perturbed_training_data])

perturbed_training_data['sentence'] = perturbed_training_data['sentence'].str.replace("\[\[","")
perturbed_training_data['sentence'] = perturbed_training_data['sentence'].str.replace("\]\]","")

perturbed_training_data = Dataset.from_pandas(perturbed_training_data)
tokenized_perturbed_training_data = perturbed_training_data.map(tokenize_function, batched=True)

<ipython-input-10-fa9ca3ae80d7>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orgininal_training_data.rename(columns={"original_text":"sentence","ground_truth_output":"label"},inplace=True)
<ipython-input-10-fa9ca3ae80d7>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  perturbed_training_data['sentence'] = perturbed_training_data['sentence'].str.replace("\[\[","")
<ipython-input-10-fa9ca3ae80d7>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  perturbed_training_data['sentence'] = perturbed_training_data['sentence'].str.replace("\]\]","")


Map:   0%|          | 0/2727 [00:00<?, ? examples/s]

Defend Model

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("ahmedrachid/FinancialBERT")

training_args = TrainingArguments(output_dir = "test_trainer",evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_perturbed_training_data,
    eval_dataset=tokenized_validation_data,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("content/drive/MyDrive/NLP Project/models/classifier_model/finbert-sentiment-defended")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ahmedrachid/FinancialBERT and are newly initialized: ['classifier.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.420062,0.840943
2,0.449400,0.497937,0.852725
3,0.136300,0.716785,0.861561


Results

In [12]:
textattack_dataset = textattack.datasets.HuggingFaceDataset(dataset['test'])

model = AutoModelForSequenceClassification.from_pretrained("content/drive/MyDrive/NLP Project/models/classifier_model/finbert-sentiment-no-defensive")
model_wrapper = HuggingFaceModelWrapper(model,tokenizer)
attack = custom_attack(model_wrapper)
attack_args = textattack.AttackArgs(
    num_examples=-1
)
attacker = textattack.Attacker(attack, textattack_dataset, attack_args)
attacks = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  NumberToWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 0 / 3 / 3:   0%|          | 3/1454 [00:00<01:17, 18.61it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (99%)]] --> [[[SKIPPED]]]

Pretax loss totaled EUR 117mn compared to a loss of EUR 65mn in the corresponding period .


--------------------------------------------- Result 2 ---------------------------------------------
[[Neutral (98%)]] --> [[[SKIPPED]]]

Business boomed after Ostrom helped plant a small story about Rapala lures in a Life magazine issue that featured Marilyn Monroe on the cover .


--------------------------------------------- Result 3 ---------------------------------------------
[[Negative (98%)]] --> [[[SKIPPED]]]

Diluted loss per share stood at EUR 0.15 versus EUR 0.26 .




[Succeeded / Failed / Skipped / Total] 0 / 3 / 3 / 6:   0%|          | 6/1454 [00:00<01:41, 14.29it/s]

...

--------------------------------------------- Result 1444 ---------------------------------------------
[[Negative (100%)]] --> [[Neutral (56%)]]

At the close , the OMX Helsinki [[25]] was [[0]].[[01]] pct lower at 3,067.64 points and the OMX Helsinki was down [[0]].05 pct at 10,321.46 points on over 1.343 bln eur of turnover .

At the close , the OMX Helsinki [[twenty-five]] was [[zero]].[[one]] pct lower at 3,067.64 points and the OMX Helsinki was down [[zero]].05 pct at 10,321.46 points on over 1.343 bln eur of turnover .


--------------------------------------------- Result 1445 ---------------------------------------------
[[Neutral (100%)]] --> [[[FAILED]]]

Seppala transferred the sale to a greater extent than last year to February , and this is reflected on the January sales figures .


--------------------------------------------- Result 1446 ---------------------------------------------
[[Neutral (99%)]] --> [[[FAILED]]]

It includes the company 's new , highly powerful , c

[Succeeded / Failed / Skipped / Total] 22 / 1193 / 237 / 1452: 100%|█████████▉| 1452/1454 [05:00<00:00,  4.83it/s]

--------------------------------------------- Result 1449 ---------------------------------------------
[[Negative (100%)]] --> [[[FAILED]]]

The air traffic of Finland has been in stoppage since then .


--------------------------------------------- Result 1450 ---------------------------------------------
[[Positive (100%)]] --> [[[SKIPPED]]]

The company 's main shareholders Lihakunta and Itikka Osuuskunta have both made a commitment to acquire shares worth EUR 20mn .


--------------------------------------------- Result 1451 ---------------------------------------------
[[Neutral (100%)]] --> [[[FAILED]]]

Sullivan said some of the boards `` really involve a lot of work , and people should get paid for their time . ''


--------------------------------------------- Result 1452 ---------------------------------------------
[[Negative (71%)]] --> [[[SKIPPED]]]

The company says it is difficult to estimate the market situation for the whole year 2009 .




[Succeeded / Failed / Skipped / Total] 22 / 1195 / 237 / 1454: 100%|██████████| 1454/1454 [05:00<00:00,  4.83it/s]

--------------------------------------------- Result 1453 ---------------------------------------------
[[Neutral (100%)]] --> [[[FAILED]]]

4 February 2011 - Finnish privacy and security software developer Tectia Oyj ( HEL : TEC1V ) said Wednesday it has set up two strategic business units - managed security and mobile authentication .


--------------------------------------------- Result 1454 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

Finnish Okmetic that manufactures and processes silicon wafers for the semiconductor and sensor industries and Norwegian solar wafer company NorSun have signed a contract under which Okmetic will supply NorSun mono silicon crystals for use in solar cell manufacturing .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 22     |
| Number of failed attacks:     | 1195   |
| Number of skipped attacks: 

In [13]:
textattack_dataset = textattack.datasets.HuggingFaceDataset(dataset['test'])

model = AutoModelForSequenceClassification.from_pretrained("content/drive/MyDrive/NLP Project/models/classifier_model/finbert-sentiment-defended")
model_wrapper = HuggingFaceModelWrapper(model,tokenizer)
attack = custom_attack(model_wrapper)
attack_args = textattack.AttackArgs(
    num_examples=-1
)
attacker = textattack.Attacker(attack, textattack_dataset, attack_args)
attacks = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  NumberToWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 1 / 2 / 3:   0%|          | 3/1454 [00:00<00:47, 30.51it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (90%)]] --> [[[FAILED]]]

Pretax loss totaled EUR 117mn compared to a loss of EUR 65mn in the corresponding period .


--------------------------------------------- Result 2 ---------------------------------------------
[[Neutral (100%)]] --> [[[SKIPPED]]]

Business boomed after Ostrom helped plant a small story about Rapala lures in a Life magazine issue that featured Marilyn Monroe on the cover .


--------------------------------------------- Result 3 ---------------------------------------------
[[Negative (100%)]] --> [[[SKIPPED]]]

Diluted loss per share stood at EUR 0.15 versus EUR 0.26 .




[Succeeded / Failed / Skipped / Total] 0 / 4 / 2 / 6:   0%|          | 6/1454 [00:00<01:24, 17.14it/s]

...

--------------------------------------------- Result 1444 ---------------------------------------------
[[Negative (100%)]] --> [[[FAILED]]]

At the close , the OMX Helsinki 25 was 0.01 pct lower at 3,067.64 points and the OMX Helsinki was down 0.05 pct at 10,321.46 points on over 1.343 bln eur of turnover .


--------------------------------------------- Result 1445 ---------------------------------------------
[[Neutral (100%)]] --> [[[FAILED]]]

Seppala transferred the sale to a greater extent than last year to February , and this is reflected on the January sales figures .


--------------------------------------------- Result 1446 ---------------------------------------------
[[Neutral (100%)]] --> [[[FAILED]]]

It includes the company 's new , highly powerful , compact and scalable Flexi Multiradio 10 Base Station .




[Succeeded / Failed / Skipped / Total] 14 / 1201 / 236 / 1451: 100%|█████████▉| 1451/1454 [04:59<00:00,  4.84it/s]

--------------------------------------------- Result 1447 ---------------------------------------------
[[Neutral (96%)]] --> [[Negative (82%)]]

At 10.58 am , Outokumpu declined 2.74 pct to [[24]].87 eur , while the OMX Helsinki 25 was 0.55 pct higher at 2,825.14 and the OMX Helsinki added [[0]].[[64]] pct to 9,386.89 .

At 10.58 am , Outokumpu declined 2.74 pct to [[twenty-four]].87 eur , while the OMX Helsinki 25 was 0.55 pct higher at 2,825.14 and the OMX Helsinki added [[zero]].[[sixty-four]] pct to 9,386.89 .


--------------------------------------------- Result 1448 ---------------------------------------------
[[Neutral (100%)]] --> [[[FAILED]]]

Automation makes it possible to conduct several tests simultaneously .


--------------------------------------------- Result 1449 ---------------------------------------------
[[Negative (100%)]] --> [[[FAILED]]]

The air traffic of Finland has been in stoppage since then .


--------------------------------------------- Result 1450 

[Succeeded / Failed / Skipped / Total] 14 / 1204 / 236 / 1454: 100%|██████████| 1454/1454 [04:59<00:00,  4.85it/s]

--------------------------------------------- Result 1452 ---------------------------------------------
[[Neutral (64%)]] --> [[[FAILED]]]

The company says it is difficult to estimate the market situation for the whole year 2009 .


--------------------------------------------- Result 1453 ---------------------------------------------
[[Neutral (100%)]] --> [[[FAILED]]]

4 February 2011 - Finnish privacy and security software developer Tectia Oyj ( HEL : TEC1V ) said Wednesday it has set up two strategic business units - managed security and mobile authentication .


--------------------------------------------- Result 1454 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

Finnish Okmetic that manufactures and processes silicon wafers for the semiconductor and sensor industries and Norwegian solar wafer company NorSun have signed a contract under which Okmetic will supply NorSun mono silicon crystals for use in solar cell manufacturing .



+---------